# Mining 'synonymous' relationships using multiple types of features

In [2]:
import json
from collections import Counter, defaultdict

import numpy as np
from sklearn.cluster import DBSCAN
from tqdm import tqdm

## Useful function for spatial features

In [3]:
def get_box_deltas(subj_boxes, obj_boxes, height, width):
    """
    Another spatial feature.
    (D(S,O), D(S,P), D(O,P)), with D(S,O)=
    (xs-xo)/ws, (ys-yo)/hs, log(ws/wo),log(hs/ho), (xo-xs)/wo, (yo-ys)/ho
    """
    pred_boxes = _compute_predicate_boxes(subj_boxes, obj_boxes)
    x_subj = (subj_boxes[:, 2] + subj_boxes[:, 3]) / 2
    y_subj = (subj_boxes[:, 0] + subj_boxes[:, 1]) / 2
    x_pred = (pred_boxes[:, 2] + pred_boxes[:, 3]) / 2
    y_pred = (pred_boxes[:, 0] + pred_boxes[:, 1]) / 2
    x_obj = (obj_boxes[:, 2] + obj_boxes[:, 3]) / 2
    y_obj = (obj_boxes[:, 0] + obj_boxes[:, 1]) / 2
    w_subj = subj_boxes[:, 3] - subj_boxes[:, 2]
    h_subj = subj_boxes[:, 1] - subj_boxes[:, 0]
    w_pred = pred_boxes[:, 3] - pred_boxes[:, 2]
    h_pred = pred_boxes[:, 1] - pred_boxes[:, 0]
    w_obj = obj_boxes[:, 3] - obj_boxes[:, 2]
    h_obj = obj_boxes[:, 1] - obj_boxes[:, 0]
    return np.stack((
        (x_subj - x_obj) / w_subj, (y_subj - y_obj) / h_subj,
        np.log(w_subj / w_obj), np.log(h_subj / h_obj),
        (x_obj - x_subj) / w_obj, (y_obj - y_subj) / h_obj,
        (x_subj - x_pred) / w_subj, (y_subj - y_pred) / h_subj,
        np.log(w_subj / w_pred), np.log(h_subj / h_pred),
        (x_pred - x_subj) / w_pred, (y_pred - y_subj) / h_pred,
        (x_obj - x_pred) / w_obj, (y_obj - y_pred) / h_obj,
        np.log(w_obj / w_pred), np.log(h_obj / h_pred),
        (x_pred - x_obj) / w_pred, (y_pred - y_obj) / h_pred,
        subj_boxes[:, 0] / height, subj_boxes[:, 1] / height,
        subj_boxes[:, 2] / width, subj_boxes[:, 3] / width,
        obj_boxes[:, 0] / height, obj_boxes[:, 1] / height,
        obj_boxes[:, 2] / width, obj_boxes[:, 3] / width,
        pred_boxes[:, 0] / height, pred_boxes[:, 1] / height,
        pred_boxes[:, 2] / width, pred_boxes[:, 3] / width,
        w_subj * h_subj / (height * width),
        w_obj * h_obj / (height * width),
        w_pred / w_subj, h_pred / h_subj,
        w_pred / w_obj, h_pred / h_obj,
        w_obj / w_subj, h_obj / h_subj
    ), axis=1)


def _compute_predicate_boxes(subj_boxes, obj_boxes):
    return np.stack([
        np.minimum(subj_boxes[:, 0], obj_boxes[:, 0]),
        np.maximum(subj_boxes[:, 1], obj_boxes[:, 1]),
        np.minimum(subj_boxes[:, 2], obj_boxes[:, 2]),
        np.maximum(subj_boxes[:, 3], obj_boxes[:, 3])
    ], axis=1)

## Load annotations

In [14]:
with open('prerequisites/sgg_annos/VRD_preddet.json') as fid:
    ANNOS = json.load(fid)
for anno in ANNOS:
    anno['objects'] = {
        'names': np.array(anno['objects']['names']),
        'ids': np.array(anno['objects']['ids']),
        'boxes': np.array(anno['objects']['boxes'])
    }
    anno['relations'] = {
        'names': np.array(anno['relations']['names']),
        'ids': np.array(anno['relations']['ids']),
        'subj_ids': np.array(anno['relations']['subj_ids']),
        'obj_ids': np.array(anno['relations']['obj_ids'])
    }
ANNOS = {anno['filename']: anno for anno in ANNOS}

with open('prerequisites/sgg_annos/' + 'VRD' + '_objects.json') as fid:
    OBJECTS = np.array(json.load(fid))
with open('prerequisites/sgg_annos/'+ 'VRD' + '_predicates.json') as fid:
    PREDICATES = np.array(json.load(fid))

INV_PRED = {pred: p for p, pred in enumerate(PREDICATES)}
INV_OBJ = {obj: o for o, obj in enumerate(OBJECTS)}

RELATIONSHIPS = sorted(list(set(
    '_'.join([subj, pred, obj])
    for anno in ANNOS.values()
    for subj, pred, obj in zip(
        np.array(anno['objects']['names'])[anno['relations']['subj_ids']],
        anno['relations']['names'],
        np.array(anno['objects']['names'])[anno['relations']['obj_ids']]
    )
)))

## Similarities using spatial features

In [15]:
def get_spatial_features(subj, obj):
    """Get spatial features for pairs with given subj.-obj."""
    spatial_features = defaultdict(list)
    for name, anno in ANNOS.items():
        inds = (
            (anno['objects']['names'][anno['relations']['subj_ids']] == subj)
            & (anno['objects']['names'][anno['relations']['obj_ids']] == obj)
        )
        if inds.any():
            subj_boxes = anno['objects']['boxes'][anno['relations']['subj_ids'][inds]]
            obj_boxes = anno['objects']['boxes'][anno['relations']['obj_ids'][inds]]
            deltas = get_box_deltas(subj_boxes, obj_boxes, anno['height'], anno['width'])
            preds = anno['relations']['names'][inds]
            for pred, delta in zip(preds, deltas):
                spatial_features[str(pred)].append(delta)
    for key in spatial_features:
        spatial_features[key] = np.array(spatial_features[key])
    return spatial_features


similarities = {}
for s, subj in tqdm(enumerate(OBJECTS)):
    for o, obj in enumerate(OBJECTS):
        # Get and sort features
        features = get_spatial_features(subj, obj)
        if not features.keys():
            continue
        names = sorted(list(features.keys()))
        relationships = [
            name for name in names for _ in range(len(features[name]))
        ]
        features = np.concatenate([features[name] for name in names], axis=0)

        # Clustering
        clustering = DBSCAN(eps=1, min_samples=1, metric='euclidean').fit(features)
        clusters = defaultdict(list)
        neg_label = 0
        for cnt, label in enumerate(clustering.labels_):
            if label > -1:
                clusters[str(label)].append(relationships[cnt])
            else:
                neg_label -= 1
                clusters[str(neg_label)].append(relationships[cnt])

        # Cluster counters
        cluster_ids = sorted([int(key) for key in clusters.keys()])
        distinct_relationships = sorted(list(set(relationships)))
        rel_counter = Counter(relationships)
        cluster_counters = []
        for cid in cluster_ids:
            counter = Counter(clusters[str(cid)])
            cluster_counters.append([counter[rel] for rel in distinct_relationships])

        # Two predicates tend to be similar if they co-exist in a cluster
        rel_sims = {}
        for r1, rel1 in enumerate(distinct_relationships):
            for r2, rel2 in enumerate(distinct_relationships):
                if rel1 == rel2:
                    rel_sims[(rel1, rel2)] = 1.0
                    continue
                rel_sims[(rel1, rel2)] = 0
                for cluster in cluster_counters:
                    if rel1 != rel2 and cluster[r1] and cluster[r2]:
                        rel_sims[(rel1, rel2)] += min(cluster[r1], cluster[r2])
                rel_sims[(rel1, rel2)] /= min(rel_counter[rel1], rel_counter[rel2])
        for (pred0, pred1) in rel_sims:
            rel0 = '_'.join([subj, pred0, obj])
            rel1 = '_'.join([subj, pred1, obj])
            if rel0 not in similarities:
                similarities[rel0] = (-np.ones(len(PREDICATES))).tolist()
                similarities[rel0][INV_PRED[pred0]] = 1.0
                similarities[rel0][-1] = 0.0
            if rel1 not in similarities:
                similarities[rel1] = (-np.ones(len(PREDICATES))).tolist()
                similarities[rel1][INV_PRED[pred1]] = 1.0
                similarities[rel1][-1] = 0.0
            similarities[rel0][INV_PRED[pred1]] = rel_sims[(pred0, pred1)]
            similarities[rel1][INV_PRED[pred0]] = rel_sims[(pred0, pred1)]
spatial_similarities = dict(similarities)
spatial_similarities['person_on_horse']

100it [04:22,  2.62s/it]


[1.0,
 -1.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 0.3333333333333333,
 -1.0,
 0.3333333333333333,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 0.0,
 0.16666666666666666,
 1.0,
 0.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.8461538461538461,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.2,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 0.0]

## Similarities using co-occurences

In [16]:
def _same_pair_annos(anno):
    """Return a list of tuples referring to a common pair."""
    relations = anno['relations']
    objects = anno['objects']['names']
    pairs = np.stack(
        (relations['subj_ids'], relations['obj_ids']), axis=1
    )
    common_pairs = (pairs[..., None] == pairs.T[None, ...]).all(1) * 1
    nz_i, nz_j = common_pairs.nonzero()
    return [
        (
            '_'.join([
                objects[relations['subj_ids'][ind_i]],
                relations['names'][ind_i],
                objects[relations['obj_ids'][ind_j]]
            ]),
            '_'.join([
                objects[relations['subj_ids'][ind_i]],
                relations['names'][ind_j],
                objects[relations['obj_ids'][ind_j]]
            ])
        )
        for ind_i, ind_j in zip(nz_i, nz_j)
        if ind_i <= ind_j
        # and relations['names'][ind_i] != relations['names'][ind_j]
    ]


same_pair_counter = Counter([
    (pair[0], pair[1])
    if pair[0] < pair[1]
    else (pair[1], pair[0])
    for anno in ANNOS.values()
    for pair in _same_pair_annos(anno)
])
rel_counter = Counter([
    '_'.join([subj, pred, obj])
    for anno in ANNOS.values()
    for subj, pred, obj in zip(
        anno['objects']['names'][anno['relations']['subj_ids']],
        anno['relations']['names'],
        anno['objects']['names'][anno['relations']['obj_ids']]
    )
])
for pair in same_pair_counter:
    same_pair_counter[pair] /= min(rel_counter[pair[0]], rel_counter[pair[1]])

similarities = {}
for pair in same_pair_counter:
    pred0 = pair[0].split('_')[1]
    pred1 = pair[1].split('_')[1]
    subj, _, obj = pair[0].split('_')
    if any(rel not in similarities for rel in pair):
        sim_vec = (-np.ones(len(PREDICATES))).tolist()
        for pred in PREDICATES:
            if '_'.join([subj, pred, obj]) in rel_counter:
                sim_vec[INV_PRED[pred]] = 0.0
        sim_vec[INV_PRED[PREDICATES[-1]]] = 0.0
    if pair[0] not in similarities:
        similarities[pair[0]] = list(sim_vec)
        similarities[pair[0]][INV_PRED[pred0]] = 1.0
    if pair[1] not in similarities:
        similarities[pair[1]] = list(sim_vec)
        similarities[pair[1]][INV_PRED[pred1]] = 1.0
    similarities[pair[0]][INV_PRED[pred1]] = min(same_pair_counter[pair], 1.0)
    similarities[pair[1]][INV_PRED[pred0]] = min(same_pair_counter[pair], 1.0)
pred_similarities = dict(similarities)
pred_similarities['person_on_horse']

[0.5,
 -1.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 0.0,
 -1.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 -1.0,
 0.3333333333333333,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.4230769230769231,
 -1.0,
 -1.0,
 0.6666666666666666,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 -1.0,
 0.0]

In [17]:
vector = spatial_similarities['person_wear_jeans']
for v, val in enumerate(vector):
    if val > -1:
        print(PREDICATES[v], val)

has 0.8674698795180723
in 0.875
near 1.0
wear 1.0
with 1.0
__background__ 0.0


In [18]:
vector = pred_similarities['person_on_horse']
# vector = spacy_similarities[0, 0, INV_PRED['wear']]
for v, val in enumerate(vector):
    if val > -1:
        print(PREDICATES[v], val)

above 0.5
adjacent to 0.0
behind 0.0
below 0.0
beside 0.0
by 0.0
has 1.0
hold 0.0
in the front of 0.0
look 0.0
near 0.0
next to 0.0
on 1.0
on the left of 0.0
on the top of 0.3333333333333333
ride 0.4230769230769231
sit on 0.6666666666666666
stand next to 0.0
taller than 0.0
touch 0.0
walk 0.0
wear 0.0
__background__ 0.0


In [19]:
assert all(key in spatial_similarities for key in RELATIONSHIPS)

In [41]:
with open('prerequisites/sgg_annos/' + 'VRD' + '_spatial_similarities.json', 'w') as fid:
    json.dump(spatial_similarities, fid)

In [11]:
with open('prerequisites/sgg_annos/' + 'VRD' + '_pred_cooccurences.json', 'w') as fid:
    json.dump(pred_similarities, fid)

In [ ]:
# plausibilities to be added 